In [56]:
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import json

In [57]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data=json.load(f)
    return data

In [58]:
hp_chars=load_data("hp_characters.json")
print(hp_chars)

['Hannah Abbott', 'Ludo Bagman', 'Bathilda Bagshot', 'Katie Bell', 'Cuthbert Binns', 'Phineas Nigellus Black', 'Sirius Black', 'Walburga Black', 'Amelia Bones', 'Susan Bones', 'Terry Boot', 'Lavender Brown', 'Millicent Bulstrode', 'Charity Burbage', 'Frank Bryce', 'Alecto Carrow', 'Amycus Carrow', 'Reginald Cattermole', 'Mary Cattermole', 'Cho Chang', 'Penelope Clearwater', 'Mrs. Cole', 'Michael Corner', 'Crabbe', 'Vincent Crabbe', 'Colin Creevey', 'Dennis Creevey', 'Dirk Cresswell', 'Barty Crouch Sr', 'Barty Crouch Jr', 'Roger Davies', 'John Dawlish', 'Fleur Delacour', 'Gabrielle Delacour', 'Dedalus Diggle', 'Amos Diggory', 'Cedric Diggory', 'Elphias Doge', 'Antonin Dolohov', 'Aberforth Dumbledore', 'Albus Dumbledore', 'Ariana Dumbledore', 'Kendra Dumbledore', 'Percival Dumbledore', 'Dudley Dursley', 'Marge Dursley', 'Petunia Dursley', 'Vernon Dursley', 'Marietta Edgecombe', 'Arabella Figg', 'Argus Filch', 'Justin Finch-Fletchley', 'Seamus Finnigan', 'Marcus Flint', 'Mundungus Fletche

In [59]:
def generate_better_characters(file):
    data=load_data(file)
    print(len(data))
    new_characters=[]
    for item in data:
        new_characters.append(item)

    for item in data:
        item=item.replace("The", "").replace("the","").replace("and","").replace("And","")
        names=item.split(" ")

        for name in names:
            name.strip()
            #print(name)
            new_characters.append(name)
        
        if "(" in item:
            names=item.split("(")
            for name in names:
                name=name.replace(")","").strip()
                new_characters.append(name)
        
        if "," in item:
            names=item.split(",")
            for name in names:
                name=name.replace("and","").strip()
                #new_characters.append(name)
            
                if " " in name:
                    new_names=name.split()
                    for x in new_names:
                        x=x.strip()
                        new_characters.append(x)
                
                new_characters.append(name)


    final_characters= []
    titles=["Dr.", "Professor", "Mr.","Mrs.", "Ms.", "Miss", "Aunt", "Uncle", "Mr. and Mrs."]
    for character in new_characters:
        if ""!=character:
            final_characters.append(character)
            for title in titles:
                titled_char=f"{title} {character}"
                final_characters.append(titled_char)
    

    final_characters=list(set(final_characters))

    print(len(final_characters))

    return final_characters


In [60]:
characters=generate_better_characters("hp_characters.json")    

208
5139


In [61]:
def create_training_data(file, type):
    data=generate_better_characters(file)
    patterns=[]
    for item in data:
        pattern = {"label":type, "pattern": item}
        patterns.append(pattern)
    return (patterns)

patterns=create_training_data("hp_characters.json","PERSON")


208
5139


In [62]:
def generate_rules(patterns):
    nlp=English()
    ruler=nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    #ruler=EntityRuler(nlp)
    #ruler.add_patterns(patterns)
    #nlp.add_pipe(ruler)
    nlp.to_disk("hp_ner")

generate_rules(patterns)

In [63]:
def test_model(model, text):
    doc=nlp(text)
    results=[]
    for ent in doc.ents:
        results.append(ent.text)
    return results

In [64]:
nlp = spacy.load("hp_ner")

In [65]:
def save_data(file, data):
    with open(file, "w", encoding="utf=8") as f:
        json.dump(data, f, indent=4)

In [66]:
ie_data={}
with open ("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", "r")as f:
    text = f.read()

    chapters = text.split("CHAPTER")[1:]
    for chapter in chapters:
        chapter_num, chapter_title = chapter.split("\n\n")[0:2]
        chapter_num = chapter_num.strip()
        segments = chapter.split("\n\n")[2:]
        hits = []
        for segment in segments:
            segment = segment.strip()
            segment = segment.replace("\n", " ")
            results = test_model(nlp, segment)
            for result in results:
                hits.append(result)
        ie_data[chapter_num] = hits

save_data("hp_data.json", ie_data)